In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
ls

In [ ]:
cd sample_data


/content/sample_data


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
input_path1 = "1.MANGO_anthracnose.csv"
input_path2 = "2.MANGO_anthracnose.csv"

input_path3 = "1.MANGO_healthy.csv"
input_path4 = "2.MANGO_healthy.csv"
input_path5 = "3.MANGO_healthy.csv"
input_path6 = "1.sooty mould.csv"

input_path7 = "2.sooty mould.csv"

input_path8 = "3.sooty mould.csv"
input_path9 = "1.MANGO_powdermildew.csv"




data1=pd.read_csv(input_path1)
data2=pd.read_csv(input_path2)
data3=pd.read_csv(input_path3)

data4=pd.read_csv(input_path4)
data5=pd.read_csv(input_path5)

data6=pd.read_csv(input_path6)
data7=pd.read_csv(input_path7)
data8=pd.read_csv(input_path8)
data9=pd.read_csv(input_path9)


In [ ]:
data1.columns

Index(['% Time Stamp', ' Integration Time', ' Reference Current', 'Diode 1',
       ' Diode 2', ' Diode 3', ' Diode 4', ' Diode 5', ' Diode 6', ' Diode 7',
       ' Diode 8', ' User Comment'],
      dtype='object')

In [ ]:
df1= data1.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)
df2= data2.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)
df3= data3.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)

df4= data4.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)


df5= data5.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)

df6= data6.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)
df7= data7.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)
df8= data8.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)
df9= data9.drop(['% Time Stamp', ' Integration Time', ' Reference Current', ' User Comment'],axis=1)



df1["label"] = 'anthracnose'
df2["label"] = 'anthracnose'
df3["label"] = 'healthy'

df4["label"] = 'healthy'

df5["label"] = 'healthy'
df6["label"] = 'sooty mould'
df7["label"] = 'sooty mould'
df8["label"] = 'sooty mould'
df9["label"] = 'powdermildew'

df1 = df1.append(df2)
df1 = df1.append(df3)

df1 = df1.append(df4)

df1 = df1.append(df5)
df1 = df1.append(df6)
df1 = df1.append(df7)
df1 = df1.append(df9)
df_data = df1.append(df9)



<ipython-input-5-4126aac3a631>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df2)
<ipython-input-5-4126aac3a631>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df3)
<ipython-input-5-4126aac3a631>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df4)
<ipython-input-5-4126aac3a631>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df5)
<ipython-input-5-4126aac3a631>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df6)
<ipython-input-5-4126aac3a631>:36: Futur

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22786 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Diode 1   22786 non-null  int64 
 1    Diode 2  22786 non-null  int64 
 2    Diode 3  22786 non-null  int64 
 3    Diode 4  22786 non-null  int64 
 4    Diode 5  22786 non-null  int64 
 5    Diode 6  22786 non-null  int64 
 6    Diode 7  22786 non-null  int64 
 7    Diode 8  22786 non-null  int64 
 8   label     22786 non-null  object
dtypes: int64(8), object(1)
memory usage: 1.7+ MB


In [ ]:
# Extracting the predictors ( Diode1 to Diode8):
X = df_data.iloc[:, 0:8].values
X

array([[ 254,  286,  306, ...,  189,  177, 6311],
       [ 267,  298,  312, ...,  189,  175, 6710],
       [ 258,  290,  308, ...,  192,  180, 6365],
       ...,
       [ 380,  377,  437, ...,  308,  310, 7895],
       [ 370,  377,  437, ...,  307,  309, 7907],
       [ 375,  373,  433, ...,  301,  303, 8017]])

In [ ]:
y = df_data.iloc[:,8].values
y=y.astype('int')
y

array([2, 2, 2, ..., 4, 4, 4])

In [ ]:
#Importing the necessary packages and libaries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
import matplotlib.pyplot as plt
import numpy as np

# Splitting the data into Training and Testing:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=0.20, random_state=80)
from sklearn import preprocessing

X_train = preprocessing.normalize(X_train)
X_test = preprocessing.normalize(X_test)

model = Sequential()
model.add(Dense(128, input_dim=8))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dropout(0.15))
model.add(Dense(5))
# this is the output for one-hot vector
model.add(Activation('softmax'))
model.summary()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# train the network
model.fit(X_train, y_train, validation_data= (X_test,y_test) , epochs=35, batch_size=16)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1152      
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 activation_1 (Activation)   (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 645       
                                                                 
 activation_2 (Activation)   (None, 5)                 0         
                                                                 
Total params: 18309 (71.52 KB)
Trainable params: 18309 (

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
54/54 [==============================] - 0s 3ms/step - loss: 0.0860 - accuracy: 0.9593
test loss, test acc: [0.08604733645915985, 0.9593329429626465]
